<a href="https://colab.research.google.com/github/sr606/Machine_Learning_CaseStudies/blob/main/Jamboree_Education_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import StandardScaler , MinMaxScaler , PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFE

from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

import statsmodels.api as sm
import statsmodels.stats.api as sms

import warnings
warnings.filterwarnings('ignore')

In [4]:
jm_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Scaler/Jamboree_Admission.csv')
df = jm_data.copy()
df.tail()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Scaler/Jamboree_Admission.csv'

In [ ]:
df = df.rename(columns={'Chance of Admit ': 'Chance_of_Admit'})

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.cov()

In [ ]:
df.describe().T

In [ ]:
df = df.drop(columns='Serial No.')
df.sample()

In [ ]:
df.columns

In [ ]:
df[df.duplicated()]

In [ ]:
df.isna().any()

In [ ]:
df.isna().sum()

In [ ]:
plt.figure(figsize=(25,8))
plt.style.use('dark_background')
sns.heatmap(df.isnull(),cmap='Greens')
plt.title('Visual Check of Nulls',fontsize=20,color='g')
plt.show()

In [ ]:
for _ in df.columns:
    print()
    print(f'Total Unique Values in {_} column are :- {df[_].nunique()}')
    print(f'Value counts in {_} column are :-\n {df[_].value_counts(normalize=True)}')
    print()
    print('-'*120)

In [ ]:
for _ in df.columns:
    print()
    print(f'Range of {_} column is from {df[_].min()} to {df[_].max()}')
    print()
    print('-'*120)

In [ ]:
df.dtypes

In [ ]:
cp = 'Greens'

In [ ]:
for _ in df.columns:
    plt.style.use('default')
    plt.style.use('fast')
    plt.figure(figsize = (18,4))
    plt.subplot(122)
    sns.boxplot(df[_],palette=cp)
    plt.subplot(121)
    sns.histplot(df[_],kde=True,color='g')
    plt.suptitle(f'Plot of {_}',fontsize=14,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
    sns.despine()
    plt.show()

In [ ]:
for col in df.columns:
    plt.figure(figsize=(25,5))
    plt.style.use('default')
    plt.style.use('fast')
    b = sns.countplot(x=df[col],palette=cp)
    plt.title(f'Distribution of {col}',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
    b.bar_label(b.containers[0], label_type='edge',fmt='%d')
    plt.xlabel(col)
    plt.ylabel('Count of Students')
    plt.tight_layout()
    sns.despine()
    plt.show();

In [ ]:
df.columns

In [ ]:
sns.pairplot(data=df, y_vars='Chance_of_Admit')
plt.show()

In [ ]:
sns.pairplot(df.corr(),y_vars='Chance_of_Admit',kind= 'reg')

In [ ]:
df.columns

In [ ]:
for col in df.columns[:-1]:
    print(col)
    sns.jointplot(data=df,x=df[col],y=df["Chance_of_Admit"],kind="reg",color='g')
    plt.show()

In [ ]:
for col in df.columns[:-1]:
    print(f"Plotting regression plot for: {col}")
    sns.regplot(data=df, x=col, y="Chance_of_Admit", color='g')
    plt.title(f'Regression Plot: {col} vs Chance_of_Admit')
    plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(), annot = True, cmap = 'Greens', linewidths = 0.1)#mask=np.triu(df.corr()))
plt.title(f'Correlations',fontsize=14,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
plt.yticks(rotation=0)
plt.show()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Initialize the LocalOutlierFactor model
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)

# Fit the model and predict outliers
outliers = lof.fit_predict(df)

# Add the outlier column to the dataframe
df['Outlier'] = outliers

# Display the outliers
outliers_df = df[df['Outlier'] == -1]
outliers_df

In [ ]:
# Function to evaluate the model
def model_evaluation(y_true, y_pred, model):
    print(f"Model: {model}")
    print(f"R2 Score: {r2_score(y_true, y_pred)}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_true, y_pred)}")
    print(f"Mean Squared Error: {mean_squared_error(y_true, y_pred)}")
    print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_true, y_pred))}")
    print()

# Remove outliers from the dataset
df_no_outliers = df[df['Outlier'] != -1]

# Statistical summary comparison
print("Statistical Summary with Outliers:")
display(df.describe())

print("\nStatistical Summary without Outliers:")
display(df_no_outliers.describe())

# Split the data into features and target variable
x_with_outliers = df.drop(columns=['Chance_of_Admit', 'Outlier'])
y_with_outliers = df['Chance_of_Admit']

x_without_outliers = df_no_outliers.drop(columns=['Chance_of_Admit', 'Outlier'])
y_without_outliers = df_no_outliers['Chance_of_Admit']

# Split the data into training and test sets
x_train_with, x_test_with, y_train_with, y_test_with = train_test_split(
    x_with_outliers, y_with_outliers, test_size=0.2, random_state=42
)
x_train_without, x_test_without, y_train_without, y_test_without = train_test_split(
    x_without_outliers, y_without_outliers, test_size=0.2, random_state=42
)

# Standardize the data
scaler_with_outliers = StandardScaler()
x_train_with = scaler_with_outliers.fit_transform(x_train_with)  # Fit and transform training data
x_test_with = scaler_with_outliers.transform(x_test_with)        # Only transform test data

scaler_without_outliers = StandardScaler()
x_train_without = scaler_without_outliers.fit_transform(x_train_without)  # Fit and transform training data
x_test_without = scaler_without_outliers.transform(x_test_without)        # Only transform test data

# Train a Linear Regression model on the data with outliers
lr_with_outliers = LinearRegression()
lr_with_outliers.fit(x_train_with, y_train_with)

# Train a Linear Regression model on the data without outliers
lr_without_outliers = LinearRegression()
lr_without_outliers.fit(x_train_without, y_train_without)

# Predict and evaluate the model performance with outliers
y_pred_train_with = lr_with_outliers.predict(x_train_with)
y_pred_test_with = lr_with_outliers.predict(x_test_with)
print("\nModel Performance with Outliers:")
model_evaluation(y_train_with, y_pred_train_with, "Linear Regression (with outliers)")
model_evaluation(y_test_with, y_pred_test_with, "Linear Regression (with outliers)")
print()

# Predict and evaluate the model performance without outliers
y_pred_train_without = lr_without_outliers.predict(x_train_without)
y_pred_test_without = lr_without_outliers.predict(x_test_without)
print("\nModel Performance without Outliers:")
model_evaluation(y_train_without, y_pred_train_without, "Linear Regression (without outliers)")
model_evaluation(y_test_without, y_pred_test_without, "Linear Regression (without outliers)")
print()

In [ ]:
x_with_outliers , y_with_outliers

In [ ]:
x_train_with , y_train_with

In [ ]:
x_test_with , y_test_with

In [ ]:
# Note : Train is fit_transformed & test is just transformed
x_train_scaled = x_train_with
y_train = y_train_with
x_test_scaled = x_test_with
y_test = y_test_with

In [ ]:
# x = scaled_df.iloc[:,:-1]
# y = scaled_df.iloc[:,-1]
print(x_train_scaled.shape , y_train.shape)
print(x_test_scaled.shape , y_test.shape)

In [ ]:
print(f'Shape of x_train: {x_train_scaled.shape}')
print(f'Shape of x_test: {x_test_scaled.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train_scaled,y_train)

In [ ]:
# Predicting values for the training and test data
y_pred_train = lr_model.predict(x_train_scaled)
y_pred_test = lr_model.predict(x_test_scaled)

In [ ]:
r2_score(y_train,y_pred_train)

In [ ]:
lr_model.score(x_train_scaled,y_train)

In [ ]:
r2_score(y_test,y_pred_test)

In [ ]:
lr_model.score(x_test_scaled,y_test)

In [ ]:
df

In [ ]:
lr_model_weights = pd.DataFrame([lr_model.coef_], columns=df.columns[:-2])
lr_model_weights['Intercept'] = lr_model.intercept_
lr_model_weights

In [ ]:
def adjusted_r2_score(y_true, y_pred, model):
    # Get the number of data points (n) and the number of features (p)
    n = len(y_true)
    p = x_train_scaled.shape[1]  # Use the number of features from training data (x_train_with)

    # Calculate R² score
    r2 = r2_score(y_true, y_pred)

    # Calculate Adjusted R²
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    return adj_r2

def model_evaluation(y_true, y_pred, model, x_train):
    print(f"Model: {model}")
    print(f"R2 Score: {r2_score(y_true, y_pred)}")
    print(f"Adjusted R2 Score: {adjusted_r2_score(y_true, y_pred, model)}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_true, y_pred)}")
    print(f"Mean Squared Error: {mean_squared_error(y_true, y_pred)}")
    print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_true, y_pred))}")
    print()



In [ ]:
print("Model Performance of training data:")
model_evaluation(y_train, y_pred_train, lr_model, x_train_scaled)

In [ ]:
print("Model Performance of test data:")
model_evaluation(y_test, y_pred_test, lr_model, x_train_scaled)

In [ ]:
new_x_train = sm.add_constant(x_with_outliers)
model = sm.OLS(y_with_outliers, new_x_train)
results = model.fit()

# statstical summary of the model
print(results.summary())

In [ ]:
new_x_train = sm.add_constant(x_train_scaled)
model = sm.OLS(y_train, new_x_train)
results = model.fit()

# statstical summary of the model
print(results.summary())

In [ ]:
vif = pd.DataFrame()
vif['Variable'] = x_with_outliers.columns
vif['VIF'] = [variance_inflation_factor(x_train_scaled, i) for i in range(x_train_scaled.shape[1])]
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Define the features and target variable for the data with outliers
x_with_outliers = df.drop(columns=['Chance_of_Admit', 'Outlier'])
y_with_outliers = df['Chance_of_Admit']

# Apply MinMax Scaling to the data with outliers
scaler_with_outliers = MinMaxScaler()
x_train_with_minmax = scaler_with_outliers.fit_transform(x_with_outliers)  # Fit and transform training data
x_test_with_minmax = scaler_with_outliers.transform(x_with_outliers)  # Only transform test data

# Convert the scaled data into a DataFrame for easier manipulation
x_train_with_minmax_df = pd.DataFrame(x_train_with_minmax, columns=x_with_outliers.columns)

# Calculate the VIF for each variable in the training set (with outliers)
vif = pd.DataFrame()
vif['Variable'] = x_train_with_minmax_df.columns
vif['VIF'] = [variance_inflation_factor(x_train_with_minmax_df.values, i) for i in range(x_train_with_minmax_df.shape[1])]
vif = vif.sort_values(by="VIF", ascending=False)

# Display the VIF values for the data with outliers
print(vif)


In [ ]:
# Filter out the outliers from the dataframe
df_no_outliers = df[df['Outlier'] != -1]

# Define features and target variable for the data with and without outliers
x_without_outliers = df_no_outliers.drop(columns=['Chance_of_Admit', 'Outlier'])
y_without_outliers = df_no_outliers['Chance_of_Admit']

x_with_outliers = df.drop(columns=['Chance_of_Admit', 'Outlier'])
y_with_outliers = df['Chance_of_Admit']

# Apply MinMax Scaling to the data with outliers
scaler_with_outliers = MinMaxScaler()
x_train_with_minmax = scaler_with_outliers.fit_transform(x_with_outliers)  # Fit and transform training data
x_test_with_minmax = scaler_with_outliers.transform(x_with_outliers)  # Only transform test data

# Convert the scaled data into a DataFrame for easier manipulation
x_train_with_minmax_df = pd.DataFrame(x_train_with_minmax, columns=x_with_outliers.columns)

# Calculate the VIF for each variable in the training set (with outliers)
vif_with_outliers = pd.DataFrame()
vif_with_outliers['Variable'] = x_train_with_minmax_df.columns
vif_with_outliers['VIF'] = [variance_inflation_factor(x_train_with_minmax_df.values, i) for i in range(x_train_with_minmax_df.shape[1])]
vif_with_outliers = vif_with_outliers.sort_values(by="VIF", ascending=False)

# Display the VIF values for the data with outliers
print("VIF for data with outliers:")
print(vif_with_outliers)

# Apply MinMax Scaling to the data without outliers
scaler_without_outliers = MinMaxScaler()
x_train_without_minmax = scaler_without_outliers.fit_transform(x_without_outliers)  # Fit and transform training data
x_test_without_minmax = scaler_without_outliers.transform(x_without_outliers)  # Only transform test data

# Convert the scaled data into a DataFrame for easier manipulation
x_train_without_minmax_df = pd.DataFrame(x_train_without_minmax, columns=x_without_outliers.columns)

# Calculate the VIF for each variable in the training set (without outliers)
vif_without_outliers = pd.DataFrame()
vif_without_outliers['Variable'] = x_train_without_minmax_df.columns
vif_without_outliers['VIF'] = [variance_inflation_factor(x_train_without_minmax_df.values, i) for i in range(x_train_without_minmax_df.shape[1])]
vif_without_outliers = vif_without_outliers.sort_values(by="VIF", ascending=False)

# Display the VIF values for the data without outliers
print("VIF for data without outliers:")
print(vif_without_outliers)


In [ ]:
residuals = y_test.values - y_pred_test

In [ ]:
residuals_train = y_train.values - y_pred_train
residuals_train.mean()

In [ ]:
residuals.mean()

In [ ]:
plt.figure(figsize=(15,5))
sns.histplot(residuals, kde= True,color='g')
plt.title('Residuals Hist',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='mediumseagreen',color='w')
sns.despine(left=True)
plt.ylabel("")
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.histplot(residuals_train, kde= True,color='g')
plt.title('Residuals Hist',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='mediumseagreen',color='w')
sns.despine(left=True)
plt.ylabel("")
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.title('Residuals_train data',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
sns.regplot(x=y_pred_train, y=residuals_train, lowess=True, color='g',line_kws={'color': 'red'})
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.subplot(122)
plt.title('Residuals_test data',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
sns.regplot(x=y_pred_test, y=residuals, lowess=True,color='g' ,line_kws={'color': 'red'})
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
sns.despine()
plt.show()

In [ ]:
# Scatterplot of residuals with each independent variable to check for Homoscedasticity
plt.figure(figsize=(15,8))
i=1
for col in x_test.columns[:-1]:
    plt.subplot(2,3,i)
    sns.scatterplot(x=x_test[col].values.reshape((-1,)), y=residuals.reshape((-1,)),color='g')
    plt.title(f'Residual Plot with {col}',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
    plt.xlabel(col)
    plt.ylabel('Residual')
    i+=1

plt.tight_layout()
sns.despine()
plt.show();

In [ ]:
ols_model = results
predicted = ols_model.predict()
residuals = ols_model.resid

In [ ]:
gq_test_results = sms.het_goldfeldquandt(residuals, ols_model.model.exog)
gq_test_results

In [ ]:
# 1. Breusch-Pagan Test for Homoscedasticity
residuals = ols_model.resid
bp_test_results = sms.het_breuschpagan(residuals, ols_model.model.exog)

# Creating DataFrame for Breusch-Pagan Test results
bp_test = pd.DataFrame(bp_test_results,
                       columns=['value'],
                       index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

# 2. Goldfeld-Quandt Test for Homoscedasticity
gq_test_results = sms.het_goldfeldquandt(residuals, ols_model.model.exog)
gq_test_df = pd.DataFrame([gq_test_results], columns=['Test Statistic', 'p-value','Trend'])

# Print Breusch-Pagan Test results
print("Breusch-Pagan Test Results:")
print(bp_test)

# Print Goldfeld-Quandt Test results
print("\nGoldfeld-Quandt Test Results:")
print(gq_test_df)

# Interpretation for Breusch-Pagan Test
bp_p_value = bp_test.loc['p-value', 'value']
if bp_p_value < 0.05:
    print("\nBreusch-Pagan Test Interpretation: Heteroscedasticity is present (p-value < 0.05).")
else:
    print("\nBreusch-Pagan Test Interpretation: Heteroscedasticity is not present (p-value >= 0.05).")

# Interpretation for Goldfeld-Quandt Test
gq_p_value = gq_test_df.loc[0, 'p-value']
if gq_p_value < 0.05:
    print("Goldfeld-Quandt Test Interpretation: Heteroscedasticity is present (p-value < 0.05).")
else:
    print("Goldfeld-Quandt Test Interpretation: Heteroscedasticity is not present (p-value >= 0.05).")



In [ ]:
# 1. White Test for Homoscedasticity
white_test_results = sms.het_white(ols_model.resid, ols_model.model.exog)

# 2. Creating DataFrame for White Test results
white_test_df = pd.DataFrame(white_test_results,
                             columns=['value'],
                             index=['Test Statistic', 'p-value', 'f-value', 'f p-value'])

# Print White Test results
print("\nWhite Test Results:")
print(white_test_df)

# Interpretation for White Test
white_p_value = white_test_df.loc['p-value', 'value']
if white_p_value < 0.05:
    print("\nWhite Test Interpretation: Heteroscedasticity is present (p-value < 0.05).")
else:
    print("\nWhite Test Interpretation: Heteroscedasticity is not present (p-value >= 0.05).")

In [ ]:
# 1. Breusch-Pagan Test for Homoscedasticity
residuals = lr_model.resid
bp_test_results = sms.het_breuschpagan(residuals, lr_model.model.exog)

# Creating DataFrame for Breusch-Pagan Test results
bp_test = pd.DataFrame(bp_test_results,
                       columns=['value'],
                       index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

# 2. Goldfeld-Quandt Test for Homoscedasticity
gq_test_results = sms.het_goldfeldquandt(residuals, lr_model.model.exog)
gq_test_df = pd.DataFrame([gq_test_results], columns=['Test Statistic', 'p-value','Trend'])

# Print Breusch-Pagan Test results
print("Breusch-Pagan Test Results:")
print(bp_test)

# Print Goldfeld-Quandt Test results
print("\nGoldfeld-Quandt Test Results:")
print(gq_test_df)

# Interpretation for Breusch-Pagan Test
bp_p_value = bp_test.loc['p-value', 'value']
if bp_p_value < 0.05:
    print("\nBreusch-Pagan Test Interpretation: Heteroscedasticity is present (p-value < 0.05).")
else:
    print("\nBreusch-Pagan Test Interpretation: Heteroscedasticity is not present (p-value >= 0.05).")

# Interpretation for Goldfeld-Quandt Test
gq_p_value = gq_test_df.loc[0, 'p-value']
if gq_p_value < 0.05:
    print("Goldfeld-Quandt Test Interpretation: Heteroscedasticity is present (p-value < 0.05).")
else:
    print("Goldfeld-Quandt Test Interpretation: Heteroscedasticity is not present (p-value >= 0.05).")


In [ ]:
plt.figure(figsize=(15,5))
sns.histplot(residuals, kde= True,color='g')
plt.title('Residuals Hist',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='mediumseagreen',color='w')
sns.despine(left=True)
plt.ylabel("")
plt.yticks([])
plt.show()

In [ ]:
# QQ-Plot of residuals
plt.figure(figsize=(15,5))
sm.qqplot(residuals,line='45')
plt.title('QQ Plot of Residuals',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
plt.ylabel('Residual Quantiles')
sns.despine()
plt.show();

In [ ]:
jb_stat, jb_p_value = stats.jarque_bera(residuals)


print("Jarque-Bera Test Statistic:", jb_stat)
print("p-value:", jb_p_value)

if jb_p_value < 0.05:
    print("Reject the null hypothesis: Residuals are not normally distributed.")
else:
    print("Fail to reject the null hypothesis: Residuals are normally distributed.")

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score

# --- LassoCV for Optimal Alpha ---
lasso_cv = LassoCV(
    alphas=np.logspace(-4, 4, 100),  # Range of alpha values
    cv=5,  # 5-fold cross-validation
    max_iter=10000,
    random_state=42
)

# Fit LassoCV on training data
lasso_cv.fit(x_train_scaled, y_train)

# Get the optimal alpha
optimal_alpha_lassocv = lasso_cv.alpha_
print(f"Optimal alpha for Lasso (LassoCV): {optimal_alpha_lassocv}")

# Evaluate the model on the test data
y_pred_lassocv = lasso_cv.predict(x_test_scaled)

# Metrics
mse = mean_squared_error(y_test, y_pred_lassocv)
r2 = r2_score(y_test, y_pred_lassocv)
print(f"Test MSE: {mse}")
print(f"Test R2 Score: {r2}")


In [ ]:
model_lasso = Lasso(alpha=0.0006428073117284319)
model_lasso.fit(x_train_scaled, y_train)

In [ ]:
model_ridge = Ridge()
model_ridge.fit(x_train_scaled, y_train)

In [ ]:
y_pred_train_ridge = model_ridge.predict(x_train_scaled)
y_pred_test_ridge = model_ridge.predict(x_test_scaled)

y_pred_train_lasso = model_lasso.predict(x_train_scaled)
y_pred_test_lasso = model_lasso.predict(x_test_scaled)

In [ ]:
lasso_model_weights = pd.DataFrame(model_lasso.coef_.reshape(1,-1),columns=df.columns[:-2])
lasso_model_weights["Intercept"] = model_lasso.intercept_
lasso_model_weights

In [ ]:
ridge_model_weights = pd.DataFrame(model_ridge.coef_.reshape(1,-1),columns=df.columns[:-2])
ridge_model_weights["Intercept"] = model_ridge.intercept_
ridge_model_weights

In [ ]:
print('Linear Regression Training Accuracy\n')
model_evaluation(y_train.values, y_pred_train, lr_model,x_train_scaled)
print('*'*25)
print('\nLinear Regression Test Accuracy\n')
model_evaluation(y_test.values, y_pred_test, lr_model,x_train_scaled)
print('---'*25)
print('\nRidge Regression Training Accuracy\n')
model_evaluation(y_train.values, y_pred_train_ridge, model_ridge,x_train_scaled)
print('*'*25)
print('\n\nRidge Regression Test Accuracy\n')
model_evaluation(y_test.values, y_pred_test_ridge, model_ridge,x_train_scaled)
print('---'*25)
print('\n\nLasso Regression Training Accuracy\n')
model_evaluation(y_train.values, y_pred_train_lasso, model_lasso,x_train_scaled)
print('*'*25)
print('\n\nLasso Regression Test Accuracy\n')
model_evaluation(y_test.values, y_pred_test_lasso, model_lasso,x_train_scaled)
print('---'*25)

In [ ]:
actual_values = y_train.values.reshape((-1,))
predicted_values = [y_pred_train.reshape((-1,)), y_pred_train_ridge.reshape((-1,)), y_pred_train_lasso.reshape((-1,))]
model = ['Linear Regression', 'Ridge Regression', 'Lasso Regression']
plt.figure(figsize=(15,5))
i=1
for preds in predicted_values:
    plt.subplot(1,3,i)
    sns.scatterplot(x=actual_values, y=preds,color='g')
    plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(model[i-1],fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
    i+=1
plt.tight_layout()
sns.despine()
plt.show();

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

# Define alpha range
alpha_range = np.logspace(-4, 4, 50)  # Small to large values of alpha

# --- LASSO with GridSearchCV ---
lasso = Lasso(max_iter=10000)
grid_lasso = GridSearchCV(
    lasso,
    param_grid={"alpha": alpha_range},
    scoring="neg_mean_squared_error",
    cv=5
)
grid_lasso.fit(x_train_scaled, y_train)

# Optimal alpha for Lasso
optimal_alpha_lasso = grid_lasso.best_params_['alpha']
print(f"Optimal alpha for Lasso (GridSearchCV): {optimal_alpha_lasso}")

# --- RIDGE with GridSearchCV ---
ridge = Ridge()
grid_ridge = GridSearchCV(
    ridge,
    param_grid={"alpha": alpha_range},
    scoring="neg_mean_squared_error",
    cv=5
)
grid_ridge.fit(x_train_scaled, y_train)

# Optimal alpha for Ridge
optimal_alpha_ridge = grid_ridge.best_params_['alpha']
print(f"Optimal alpha for Ridge (GridSearchCV): {optimal_alpha_ridge}")

# --- RandomizedSearchCV ---
random_lasso = RandomizedSearchCV(
    lasso,
    param_distributions={"alpha": alpha_range},
    n_iter=20,
    scoring="neg_mean_squared_error",
    cv=5,
    random_state=42
)
random_lasso.fit(x_train_scaled, y_train)
optimal_alpha_lasso_random = random_lasso.best_params_['alpha']
print(f"Optimal alpha for Lasso (RandomizedSearchCV): {optimal_alpha_lasso_random}")

random_ridge = RandomizedSearchCV(
    ridge,
    param_distributions={"alpha": alpha_range},
    n_iter=20,
    scoring="neg_mean_squared_error",
    cv=5,
    random_state=42
)
random_ridge.fit(x_train_scaled, y_train)
optimal_alpha_ridge_random = random_ridge.best_params_['alpha']
print(f"Optimal alpha for Ridge (RandomizedSearchCV): {optimal_alpha_ridge_random}")

# --- AIC / BIC Method ---
def calculate_aic_bic(model, x, y):
    # Fit the model
    model.fit(x, y)
    predictions = model.predict(x)
    residual_sum_of_squares = mean_squared_error(y, predictions) * len(y)
    n, p = x.shape  # Number of samples, number of predictors
    aic = n * np.log(residual_sum_of_squares / n) + 2 * p
    bic = n * np.log(residual_sum_of_squares / n) + p * np.log(n)
    return aic, bic

# Calculate AIC / BIC for Lasso
aic_lasso, bic_lasso = calculate_aic_bic(Lasso(alpha=optimal_alpha_lasso), x_test_scaled, y_test)
print(f"AIC for Lasso: {aic_lasso}, BIC for Lasso: {bic_lasso}")

# Calculate AIC / BIC for Ridge
aic_ridge, bic_ridge = calculate_aic_bic(Ridge(alpha=optimal_alpha_ridge), x_test_scaled, y_test)
print(f"AIC for Ridge: {aic_ridge}, BIC for Ridge: {bic_ridge}")

# --- Final Testing with Optimal Alphas ---
lasso_final = Lasso(alpha=optimal_alpha_lasso)
lasso_final.fit(x_train_scaled, y_train)
lasso_test_score = lasso_final.score(x_test_scaled, y_test)
print(f"Lasso Test Score with optimal alpha: {lasso_test_score}")

ridge_final = Ridge(alpha=optimal_alpha_ridge)
ridge_final.fit(x_train_scaled, y_train)
ridge_test_score = ridge_final.score(x_test_scaled, y_test)
print(f"Ridge Test Score with optimal alpha: {ridge_test_score}")


In [ ]:
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score

# --- ElasticNetCV for Optimal Alpha and L1 Ratio ---
elasticnet_cv = ElasticNetCV(
    alphas=np.logspace(-4, 4, 100),  # Range of alpha values
    l1_ratio=np.linspace(0.1, 1.0, 10),  # Range of l1_ratio values
    cv=5,  # 5-fold cross-validation
    max_iter=10000,
    random_state=42
)

# Fit ElasticNetCV on training data
elasticnet_cv.fit(x_train_scaled, y_train)

# Get the optimal alpha and l1_ratio
optimal_alpha_elasticnet = elasticnet_cv.alpha_
optimal_l1_ratio_elasticnet = elasticnet_cv.l1_ratio_
print(f"Optimal alpha for ElasticNet: {optimal_alpha_elasticnet}")
print(f"Optimal l1_ratio for ElasticNet: {optimal_l1_ratio_elasticnet}")

# Evaluate the model on the test data
y_pred_elasticnet = elasticnet_cv.predict(x_test_scaled)

# Metrics
mse = mean_squared_error(y_test, y_pred_elasticnet)
r2 = r2_score(y_test, y_pred_elasticnet)
print(f"Test MSE: {mse}")
print(f"Test R2 Score: {r2}")


In [ ]:
# ElasticNet_model = ElasticNet(alpha=0.108)
# ElasticNet_model.fit(x_train , y_train)
ElasticNet_model = ElasticNet(alpha=0.108)
ElasticNet_model.fit(x_train_scaled , y_train)

In [ ]:
y_pred_train_en = ElasticNet_model.predict(x_train_scaled)
y_pred_test_en = ElasticNet_model.predict(x_test_scaled)

In [ ]:
train_R2 = ElasticNet_model.score(x_train_scaled,y_train)
test_R2 = ElasticNet_model.score(x_test_scaled,y_test)
train_R2 , test_R2

In [ ]:
en_model_weights = pd.DataFrame(ElasticNet_model.coef_.reshape(1,-1),columns=df.columns[:-2])
en_model_weights["Intercept"] = ElasticNet_model.intercept_
en_model_weights

In [ ]:
print('ElasticNet Regression Training Accuracy\n')
model_evaluation(y_train.values, y_pred_train_en, ElasticNet_model,x_train_scaled)
print('*'*25)
print('\nElasticNet Regression Test Accuracy\n')
model_evaluation(y_test.values, y_pred_test_en, ElasticNet_model,x_train_scaled)
print('---'*25)

In [ ]:
actual_values = y_train.values.reshape((-1,))
predicted_values = [y_pred_train.reshape((-1,)), y_pred_train_ridge.reshape((-1,)),
                    y_pred_train_lasso.reshape((-1,)),y_pred_train_en.reshape((-1,))]
model = ['Linear Regression', 'Ridge Regression', 'Lasso Regression','ElasticNet Regression']
plt.figure(figsize=(15,4))
i=1
for preds in predicted_values:
    plt.subplot(1,4,i)
    sns.scatterplot(x=actual_values, y=preds,color='g')
    plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(model[i-1],fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
    i+=1
plt.tight_layout()
sns.despine()
plt.show();

In [ ]:
model_major_weights = {"Linear Model":lr_model_weights,
                       "Ridge Model":ridge_model_weights,
                       "Lasso Model":lasso_model_weights,
                       "Elastic_Net":en_model_weights}
# excluding w0-intercept
plt.figure(figsize=(25,5))
i=1
for model,data in model_major_weights.items():
    model_weights_data = data.melt()

    plt.subplot(1,4,i)
    sns.barplot(data=model_weights_data[:-1].sort_values(by='value',ascending=False),
                y='variable', x='value',width=0.2,palette=['darkgreen','g','green','limegreen','seagreen','mediumseagreen'])
    plt.xlabel('Features')
    plt.ylabel('Coefficient/Weight')
    plt.title(f'{model} Coefficients / weights',fontsize=12,fontfamily='serif',fontweight='bold',backgroundcolor='g',color='w')
    i+=1
sns.despine()
plt.show()

In [ ]:
# Function for Adj. R2 Score
def adj_r(r_sq,X,Y):
  adj_r1 = (1 - ((1-r_sq)*(len(Y)-1)) / (len(Y)-X.shape[1]-1))
  return adj_r1

def r2_score(y,y_):
    num = np.sum((y-y_)**2)
    denom = np.sum((y- y.mean())**2)
    score = (1- num/denom)
    return score

In [ ]:
# Creating a pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

degrees = 4
train_scores = []
test_scores = []

train_loss = []
test_loss = []

for degree in range(1, degrees):
    # Putting the classes like PolynomialFeatures(), StandardScaler(), LinearRegression() into a pipeline
    polyreg_scaled = make_pipeline(PolynomialFeatures(degree), StandardScaler(), LinearRegression())
    polyreg_scaled.fit(x_train_scaled, y_train)

    # Calculate R2 Score for train and test data
    train_score = polyreg_scaled.score(x_train_scaled, y_train) # R2 TRAIN
    test_score = polyreg_scaled.score(x_test_scaled, y_test) # R2 TEST

    # Calculate Adj. R2 Score for train and test data
    train_scores.append(adj_r(train_score,x_train_scaled,y_train))
    test_scores.append(adj_r(test_score,x_test_scaled,y_test))

    # Calculate the y_pred for train and test data
    output1 = polyreg_scaled.predict(x_train_scaled)
    output2 = polyreg_scaled.predict(x_test_scaled)

    # Calculate the MSE for train and test data
    train_loss.append(mean_squared_error(y_train,output1)) # MSE train
    test_loss.append(mean_squared_error(y_test,output2)) # MSE test

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Plot Adjusted R-scores
axes[0].plot(list(range(1, degrees)), train_scores, label="train")
axes[0].plot(list(range(1, degrees)), test_scores, label="test")
axes[0].axvline(x=2, color='g', linestyle='--', label="degree=2")
axes[0].legend(loc='lower left')
axes[0].set_xlabel("Degree")
axes[0].set_ylabel("Adj. R² Score")
axes[0].set_title("Adjusted R² Score vs. Degree", fontsize=12, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')

# Plot Mean Squared Errors
axes[1].plot(list(range(1, degrees)), train_loss, label="train")
axes[1].plot(list(range(1, degrees)), test_loss, label="test")
axes[1].axvline(x=2, color='g', linestyle='--', label="degree=2")
axes[1].legend(loc='upper left')
axes[1].set_xlabel("Degree")
axes[1].set_ylabel("MSE")
axes[1].set_title("Mean Squared Error vs. Degree", fontsize=12, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')

sns.despine()
plt.tight_layout()
plt.show()


In [ ]:
rate_list = list(range(1, degrees))

In [ ]:
# Best degree
index = np.argmax(test_scores)
best_degree = rate_list[index]
best_degree

In [ ]:
# Polynomial Regression

# Transform the features into polynomial features
from sklearn.preprocessing import PolynomialFeatures
degree = 2
poly = PolynomialFeatures(degree=degree)
X_train_poly = poly.fit_transform(x_train_scaled)
X_test_poly = poly.transform(x_test_scaled)

In [ ]:
# Shape of X_train and X_test
x_train_scaled.shape, x_test_scaled.shape

In [ ]:
# Polynomial features been created
X_train_poly.shape, X_test_poly.shape

In [ ]:
# Standardize the polynomial features

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

In [ ]:
# Import the required library
from sklearn.linear_model import LinearRegression

# Initialize the Linear Regression model
Polynomial_Reg_model = LinearRegression()

# Train the model
Polynomial_Reg_model.fit(X_train_poly_scaled, y_train)

In [ ]:
# Predicting values for the test data
y_pred_train_poly = Polynomial_Reg_model.predict(X_train_poly_scaled)
y_pred_test_poly = Polynomial_Reg_model.predict(X_test_poly_scaled)

In [ ]:
# Performance of Polynomial Regression
print("Performance of Polynomial Regression")
print("-"*36)
print("Performance of Train data")
print("-"*26)
model_evaluation(y_train, y_pred_train_poly, Polynomial_Reg_model,x_test_scaled)
print()
print("Performance of Test data")
print("-"*26)
model_evaluation(y_test, y_pred_test_poly, Polynomial_Reg_model,x_test_scaled)

In [ ]:
# Actual vs Predicted
plt.figure(figsize=(14, 5))

# Actual vs Predicted Plot
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_pred_train_poly,color="green")
plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
plt.xlabel('Actual_train (y_train)')
plt.ylabel('Predicted_train (y_pred_train)')
plt.title('Training data')

# Actual vs Predicted Plot
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_test_poly, color="green")
plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
plt.xlabel('Actual_test (y_test)')
plt.ylabel('Predicted_test (y_pred_test)')
plt.title('Testing data')

plt.suptitle("Actual vs Predicted - Polynomial Regression", fontsize = 15, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')
sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
X_train = x_train_scaled
X_test = x_test_scaled

In [ ]:
# Hyperparameter Tuning: find the best regularization strength
from sklearn.linear_model import Lasso, Ridge

# To find best lambda
degree = 2 # is best

train_scores = []
test_scores = []

rate_list = np.linspace(0,0.003,50)

for rate in rate_list:

  # Creating pipeline()
  polyreg_scaled = make_pipeline(PolynomialFeatures(2), StandardScaler(), Lasso(alpha=rate))
  polyreg_scaled.fit(X_train, y_train)

  # Calculate R2 Score for train and test data
  train_score = polyreg_scaled.score(X_train, y_train)
  test_score = polyreg_scaled.score(X_test, y_test)

  # Calculate Adj. R2 Score for train and test data
  train_scores.append(adj_r(train_score,X_train,y_train))
  test_scores.append(adj_r(test_score,X_test,y_test))


In [ ]:
# Best Lambda (or) alpha
index = np.argmax(test_scores)
best_lambda = rate_list[index]
best_lambda

In [ ]:
plt.figure(figsize=(16, 5))

# Plot train and test scores
plt.plot(rate_list, train_scores, label="train")
plt.plot(rate_list, test_scores, label="val")

# Scatter plot for the best lambda
plt.scatter(best_lambda + 0.000034, test_scores[index], color='g', s=100, label=f"Best λ = {best_lambda:.4f}")
# plt.annotate(f"Best λ = {best_lambda:.4f}",
#              xy=(best_lambda, test_scores[index]),
#              xytext=(best_lambda + 0.0001, test_scores[index] - 0.02),
#              arrowprops=dict(facecolor='red', arrowstyle='->'),
#              fontsize=12, color='red')

# Styling and labels
plt.legend(loc='lower right')
plt.xlabel("lambda")
plt.ylabel("R² Score")
plt.title("Polynomial Regression - Hyperparameter Tuning",
          fontsize=15, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')
plt.grid()
sns.despine()

plt.show()


In [ ]:
# Final Lasso model
# degree 2 and lambda :0.0019591836734693877
final_lasso_model_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), Lasso(alpha=best_lambda))
final_lasso_model_pipe.fit(X_train, y_train)

# Predicting values for the train and test data
y_pred_train_lasso = final_lasso_model_pipe.predict(X_train)
y_pred_test_lasso = final_lasso_model_pipe.predict(X_test)

# Performance of Lasso Regression
print("Performance of Lasso Regression")
print("-"*36)
# Metrix for train and test data
print("Performance of Train data")
print("-"*26)
model_evaluation(y_train, y_pred_train_lasso, final_lasso_model_pipe,x_train_scaled)
print()
print("Performance of Test data")
print("-"*26)
model_evaluation(y_test, y_pred_test_lasso, final_lasso_model_pipe,x_train_scaled)

In [ ]:
# Actual vs Predicted
plt.figure(figsize=(14, 5))

# Actual vs Predicted Plot
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_pred_train_lasso,color="g")
plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
plt.xlabel('Actual_train (y_train)')
plt.ylabel('Predicted_train (y_pred_train)')
plt.title('Training data')

# Actual vs Predicted Plot
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_test_lasso, color="green")
plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
plt.xlabel('Actual_test (y_test)')
plt.ylabel('Predicted_test (y_pred_test)')
plt.title('Testing data')

plt.suptitle("Actual vs Predicted - Lasso Polynomial Regression", fontsize = 15, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')
sns.despine()
plt.tight_layout()
plt.show()


In [ ]:
# Hyperparameter Tuning: find the best regularization strength
from sklearn.linear_model import Lasso, Ridge

# To find best lambda
degree = 2 # is best

train_scores = []
test_scores = []

rate_list = np.linspace(0,0.0002,10)

for rate in rate_list:

  # Creating pipeline()
  polyreg_scaled = make_pipeline(PolynomialFeatures(2), StandardScaler(), Ridge(alpha=rate))
  polyreg_scaled.fit(X_train, y_train)

  # Calculate R2 Score for train and test data
  train_score = polyreg_scaled.score(X_train, y_train)
  test_score = polyreg_scaled.score(X_test, y_test)

  # Calculate Adj. R2 Score for train and test data
  train_scores.append(adj_r(train_score,X_train,y_train))
  test_scores.append(adj_r(test_score,X_test,y_test))

# Plote
plt.figure(figsize=(14, 5))
plt.plot(rate_list, train_scores, label="train")
plt.plot(rate_list, test_scores, label="val")
plt.legend(loc='lower right')
plt.xlabel("lambda")
plt.ylabel("R-score")
plt.grid()
plt.show()

In [ ]:
# Best Lambda (or) alpha
index = np.argmax(test_scores)
best_lambda_Ridge = rate_list[index]
best_lambda_Ridge

In [ ]:
plt.figure(figsize=(16, 5))

# Plot train and test scores
plt.plot(rate_list, train_scores, label="train")
plt.plot(rate_list, test_scores, label="val")

# Scatter plot for the best lambda
plt.scatter(best_lambda_Ridge, test_scores[index], color='g', s=100, label=f"Best λ = {best_lambda_Ridge:.6f}")
# plt.annotate(f"Best λ = {best_lambda:.4f}",
#              xy=(best_lambda, test_scores[index]),
#              xytext=(best_lambda + 0.0001, test_scores[index] - 0.02),
#              arrowprops=dict(facecolor='red', arrowstyle='->'),
#              fontsize=12, color='red')

# Styling and labels
plt.legend(loc='lower right')
plt.xlabel("lambda")
plt.ylabel("R² Score")
plt.title("Polynomial Regression - Hyperparameter Tuning",
          fontsize=15, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')
plt.grid()
sns.despine()

plt.show()


In [ ]:
# Final Lasso model
# degree: 2 and lambda :2.2222222222222223e-05
final_ridge_model_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), Ridge(alpha=best_lambda_Ridge))
final_ridge_model_pipe.fit(X_train, y_train)

# Predicting values for the train and test data
y_pred_train_ridge = final_ridge_model_pipe.predict(X_train)
y_pred_test_ridge = final_ridge_model_pipe.predict(X_test)

# Performance of Ridge Regression
print("Performance of Ridge Regression")
print("-"*36)
# Metrix for train and test data
print("Performance of Train data")
print("-"*26)
model_evaluation(y_train, y_pred_train_ridge, final_ridge_model_pipe,x_train_scaled)
print()
print("Performance of Test data")
print("-"*26)
model_evaluation(y_test, y_pred_test_ridge, final_ridge_model_pipe,x_train_scaled)

In [ ]:
# Actual vs Predicted
plt.figure(figsize=(16, 5))

# Actual vs Predicted Plot
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_pred_train_ridge,color="g")
plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
plt.xlabel('Actual_train (y_train)')
plt.ylabel('Predicted_train (y_pred_train)')
plt.title('Training data')

# Actual vs Predicted Plot
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_test_ridge, color="green")
plt.plot([np.min(actual_values), np.max(actual_values)], [np.min(actual_values), np.max(actual_values)], 'r-')
plt.xlabel('Actual_test (y_test)')
plt.ylabel('Predicted_test (y_pred_test)')
plt.title('Testing data')

plt.suptitle("Actual vs Predicted - RIDGE Polynomial Regression", fontsize = 15, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')
sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
# Hyperparameter Tuning: find the best regularization strength
from sklearn.linear_model import ElasticNet

# To find best lambda
degree = 2 # is best

train_scores = []
test_scores = []

rate_list = np.linspace(0,0.006,10)

for rate in rate_list:

  # Creating pipeline()
  polyreg_scaled = make_pipeline(PolynomialFeatures(2), StandardScaler(), ElasticNet(alpha=rate))
  polyreg_scaled.fit(X_train, y_train)

  # Calculate R2 Score for train and test data
  train_score = polyreg_scaled.score(X_train, y_train)
  test_score = polyreg_scaled.score(X_test, y_test)

  # Calculate Adj. R2 Score for train and test data
  train_scores.append(adj_r(train_score,X_train,y_train))
  test_scores.append(adj_r(test_score,X_test,y_test))

  # Best Lambda (or) alpha
index = np.argmax(test_scores)
best_lambda_ElasticNet = rate_list[index]
print(f"best_lambda_ElasticNet : {best_lambda_ElasticNet}")


plt.figure(figsize=(16, 5))

# Plot train and test scores
plt.plot(rate_list, train_scores, label="train")
plt.plot(rate_list, test_scores, label="val")

# Scatter plot for the best lambda
plt.scatter(best_lambda_ElasticNet, test_scores[index], color='g', s=100, label=f"Best λ = {best_lambda_Ridge:.6f}")
# plt.annotate(f"Best λ = {best_lambda:.4f}",
#              xy=(best_lambda, test_scores[index]),
#              xytext=(best_lambda + 0.0001, test_scores[index] - 0.02),
#              arrowprops=dict(facecolor='red', arrowstyle='->'),
#              fontsize=12, color='red')

# Styling and labels
plt.legend(loc='lower right')
plt.xlabel("lambda")
plt.ylabel("R² Score")
plt.suptitle("Actual vs Predicted - ELASTICNET Polynomial Regression", fontsize = 15, fontfamily='serif', fontweight='bold', backgroundcolor='g', color='w')
plt.grid()
sns.despine()

plt.show()


In [ ]:
# Final ElasticNet model
# degree: 2 and lambda :0.004
final_ElasticNet_model_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), Ridge(alpha=best_lambda_ElasticNet))
final_ElasticNet_model_pipe.fit(X_train, y_train)

# Predicting values for the train and test data
y_pred_train_ElasticNet = final_ElasticNet_model_pipe.predict(X_train)
y_pred_test_ElasticNet = final_ElasticNet_model_pipe.predict(X_test)

# Performance of ElasticNet Regression
print("Performance of ElasticNet Regression")
print("-"*36)
# Metrix for train and test data
print("Performance of Train data")
print("-"*26)
model_evaluation(y_train, y_pred_train_ElasticNet, final_ElasticNet_model_pipe,x_test_scaled)
print()
print("Performance of Test data")
print("-"*26)
model_evaluation(y_test, y_pred_test_ElasticNet, final_ElasticNet_model_pipe,x_test_scaled)